# Cython and Numba

In [1]:
import Cython
%load_ext Cython

# Note: To use Cython on a Windows machine you may need to install Visual Studio to use it 
# to install the Python Extensions Package

In [2]:
from array import array
from random import randint

In [3]:
arr = array('i', (randint(-1000, 1000) for _ in range(10000000)))

In [4]:
def sum_py(arr):
    sum_ = 0
    for elem in arr:
        sum_ += elem
    return sum_

In [5]:
%timeit -r3 -n5 sum(arr)
%timeit -r3 -n5 sum_py(arr)

205 ms ± 8.04 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)
451 ms ± 39.8 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


In [6]:
%%cython --annotate
def sum_cy(arr):
    sum_ = 0
    for elem in arr:
        sum_ += elem
    return sum_

In [7]:
%timeit -r3 -n5 sum(arr)
%timeit -r3 -n5 sum_cy(arr)

199 ms ± 14.6 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)
351 ms ± 13.2 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


### Static Typing

In [8]:
%%cython
def sum_cy_st(arr):
    cdef int sum_ = 0
    for elem in arr:
        sum_ += elem
    return sum_

In [9]:
%timeit -r3 -n5 sum(arr)
%timeit -r3 -n5 sum_cy_st(arr)

178 ms ± 9.88 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)
491 ms ± 39.7 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


#### Typed Memoryviews

In [10]:
%%cython
def sum_cy_tm_1(int[:] arr):
    cdef int sum_ = 0
    for elem in arr:
        sum_ += elem
    return sum_

In [11]:
%timeit -r3 -n5 sum(arr)
%timeit -r3 -n5 sum_cy_st(arr)
%timeit -r2 -n3 sum_cy_tm_1(arr)

208 ms ± 2.42 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)
466 ms ± 19.1 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)
435 ms ± 346 μs per loop (mean ± std. dev. of 2 runs, 3 loops each)


### With C-like looping

In [12]:
%%cython
from cpython cimport array
def sum_cy_tm_2(int[:] arr):
    cdef int i, n = arr.shape[0], sum_ = 0
    for i in range(n):
        sum_ += arr[i]
    return sum_

In [13]:
%timeit -r3 -n5 sum(arr)
%timeit -r3 -n5 sum_cy_st(arr)
%timeit -r3 -n5 sum_cy_tm_2(arr)

180 ms ± 8.59 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)
449 ms ± 8.51 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)
14.8 ms ± 2.48 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)


### Compile Optimizations

In [14]:
%%cython
cimport cython
from cpython cimport array

@cython.boundscheck(False)  # just read from/write to memory without checking if an index is in a given range
@cython.wraparound(False)  # using negative indices is not possible anymore
# see also: https://docs.cython.org/en/latest/src/userguide/source_files_and_compilation.html#compiler-directives
def sum_cy_co(int[::1] arr):
    cdef int i, n = arr.shape[0], sum_ = 0
    for i in range(n):
        sum_ += arr[i]
    return sum_

In [15]:
%timeit -r3 -n5 sum(arr)
%timeit -r3 -n5 sum_cy_co(arr)

205 ms ± 12.5 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)
4.31 ms ± 787 μs per loop (mean ± std. dev. of 3 runs, 5 loops each)


In [16]:
# With Memory view layout --> int[::1] when passing an array as function argument
# see also: https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html
%timeit -r3 -n5 sum(arr)
%timeit -r3 -n5 sum_cy_co(arr)

211 ms ± 8.87 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)
4.91 ms ± 791 μs per loop (mean ± std. dev. of 3 runs, 5 loops each)


### Together with Multithreading

In [17]:
%%cython
cimport cython
from cpython cimport array

@cython.boundscheck(False)
@cython.wraparound(False)
def _sum_gil(int[::1] arr, int[:] out):
    cdef int i, n = arr.shape[0], sum_ = 0
    
    #with nogil:
    for i in range(n):
        sum_ += arr[i]
        
    out[0] = sum_
      
def sum_threads(int[::1] arr):
    cdef int[::1] arr1 = arr[0 : arr.shape[0]//2]
    cdef int[::1] arr2 = arr[arr.shape[0]//2 : arr.shape[0]]
    cdef int[:] out = array.array('i', (0, 0))
    
    from threading import Thread
    t1 = Thread(target=_sum_gil, args=(arr1, out[0:1]))
    t2 = Thread(target=_sum_gil, args=(arr2, out[1:2]))
    
    t1.start(); t2.start()
    t1.join(); t2.join()
    
    return out[0] + out[1]

In [18]:
# with GIL
%timeit -r3 -n5 sum(arr)
%timeit -r3 -n5 sum_threads(arr)

184 ms ± 7.36 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)
6.69 ms ± 239 μs per loop (mean ± std. dev. of 3 runs, 5 loops each)


In [19]:
# with GIL released (use 'with nogil' statement)
%timeit -r3 -n5 sum(arr)
%timeit -r3 -n5 sum_threads(arr)

182 ms ± 12.5 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)
6.18 ms ± 512 μs per loop (mean ± std. dev. of 3 runs, 5 loops each)


### Numpy (Vecotrization)

In [20]:
import numpy as np

np_arr = np.asarray(arr)

%timeit -r3 -n5 sum(np_arr)
%timeit -r3 -n5 np_arr.sum()
%timeit -r3 -n5 sum_cy_co(np_arr)

530 ms ± 27.3 ms per loop (mean ± std. dev. of 3 runs, 5 loops each)
9.02 ms ± 39.2 μs per loop (mean ± std. dev. of 3 runs, 5 loops each)
3.46 ms ± 60.3 μs per loop (mean ± std. dev. of 3 runs, 5 loops each)


### Proof that they are all working

In [21]:
(sum_py(arr), sum_cy(arr), sum(arr), sum_cy_st(arr), sum_cy_tm_1(arr), sum_cy_tm_2(arr), sum_cy_co(arr), sum_threads(arr), np_arr.sum())

(-895675,
 -895675,
 -895675,
 -895675,
 -895675,
 -895675,
 -895675,
 -895675,
 np.int64(-895675))

## Numba

In [22]:
import os
os.environ['NUMBA_NUM_THREADS'] = '10'
import numba as nb
import numpy as np

In [23]:
arr_nb = np.random.random((4096,4096))

In [24]:
def py_sum2d(arr):
    result = 0.0
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            result += arr[i, j]
    return result

@nb.jit
def nb_sum2d(arr):
    result = 0.0
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            result += arr[i, j]
    return result

In [25]:
%timeit -r1 -n1 py_sum2d(arr_nb)
%timeit -r1 -n1 nb_sum2d(arr_nb)  # --> first call is slow

2.95 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
546 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [26]:
%timeit -r1 -n1 py_sum2d(arr_nb)
%timeit -r1 -n1 nb_sum2d(arr_nb)  # --> first call is slow

2.69 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
24.4 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [27]:
@nb.jit(nopython=True)
#@nb.njit
def nb_sum2d_nopython(arr):
    result = 0.0
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            result += arr[i, j]
    return result

In [28]:
%timeit -r5 -n10 nb_sum2d(arr_nb)
%timeit -r5 -n10 nb_sum2d_nopython(arr_nb)

26.2 ms ± 3.63 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)
25.8 ms ± 4.71 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [29]:
@nb.jit(nopython=True, parallel=True)
def nb_sum2d_parallel(arr):
    result = 0.0
    for i in nb.prange(arr.shape[0]):
        for j in nb.prange(arr.shape[1]):
            result += arr[i, j]
    return result

In [30]:
%timeit -r5 -n10 nb_sum2d(arr_nb)
%timeit -r5 -n10 nb_sum2d_parallel(arr_nb)  # --> add nb.prange (https://numba.pydata.org/numba-doc/0.11/prange.html)

24.8 ms ± 2.8 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)
The slowest run took 26.75 times longer than the fastest. This could mean that an intermediate result is being cached.
19.5 ms ± 31.9 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [31]:
nb_sum2d_parallel.parallel_diagnostics(level=1)

 
 Parallel Accelerator Optimizing:  Function nb_sum2d_parallel, 
C:\Users\eq05ukog\AppData\Local\Temp\ipykernel_5348\3135309123.py (1)  


Parallel loop listing for  Function nb_sum2d_parallel, C:\Users\eq05ukog\AppData\Local\Temp\ipykernel_5348\3135309123.py (1) 
---------------------------------------------|loop #ID
@nb.jit(nopython=True, parallel=True)        | 
def nb_sum2d_parallel(arr):                  | 
    result = 0.0                             | 
    for i in nb.prange(arr.shape[0]):--------| #1
        for j in nb.prange(arr.shape[1]):----| #0
            result += arr[i, j]              | 
    return result                            | 
------------------------------ After Optimisation ------------------------------
Parallel region 0:
+--1 (parallel)
   +--0 (serial)


 
Parallel region 0 (loop #1) had 0 loop(s) fused and 1 loop(s) serialized as part
 of the larger parallel loop (#1).
--------------------------------------------------------------------------------
-----